# Open Graph Benchmark (OGB) Node Prediction for Micrososoft Academic Graph (OGBN-MAG)

The [Open Graph Benchmark (OGB)](https://ogb.stanford.edu/) is a collection of realistic, large-scale, and diverse benchmark datasets for machine learning on graphs.

This notebook demonstrate a [Graph Neural Network ([GNN])](https://en.wikipedia.org/wiki/Graph_neural_network) based on [TF-GNN](https://github.com/tensorflow/gnn) [OGBNMAG tutorial](https://colab.research.google.com/github/tensorflow/gnn/blob/master/examples/notebooks/ogbn_mag_e2e.ipynb), but using GoMLX.

The task [OGBN-MAG is described here](https://ogb.stanford.edu/docs/nodeprop/#ogbn-mag). This demo comes with a library that does the downloading, parsing and converting of the data to tensors for fast use.

The model is experimental, but it includes a basic GNN library that can be used for different projects.

**EXPERIMENTAL**, it has been used only for OGBN-MAG still.

See also [OGBN-MAG Leaderboard](https://ogb.stanford.edu/docs/leader_nodeprop/#ogbn-papers100M) -- take with a grain of salt because different models use different tricks that may be considered leaking, or using extra data from the outside (so more data), and in some cases are very overfit to the task. But still, it's a fun dataset to work with.

See the subdirectory `demo` for a command line of the trainer (that can be run on the cloud somewhere) and it will save datapoints that can be plotted in the notebook later.


In [1]:
!*rm -f go.work && go work init && go work use . "${HOME}/Projects/gomlx" "${HOME}/Projects/gonb"
%goworkfix

	- Added replace rule for module "github.com/gomlx/gomlx" to local directory "/home/janpf/Projects/gomlx".
	- Added replace rule for module "github.com/janpfeifer/gonb" to local directory "/home/janpf/Projects/gonb".


## Downloading Dataset

The method `mag.Download()` will download the dataset to the data directory, if not yet downloaded.
It then converts the dataset to tensors, which are then available for use.

The tensor is then saved for faster access. After saved, the next call to `mag.Download()` will take 1/2s.

In [3]:
import (
    "flag"
    mag "github.com/gomlx/gomlx/examples/ogbnmag"
    "github.com/janpfeifer/must"
    . "github.com/gomlx/gomlx/graph"
	"github.com/gomlx/gomlx/ml/context"
)

var (
    flagDataDir   = flag.String("data", "~/work/ogbnmag", "Directory to cache downloaded and generated dataset files.")
    
    manager = NewManager()
)

%%
start := time.Now()
must.M(mag.Download(*flagDataDir))
magSampler := must.M1(mag.NewSampler(*flagDataDir))
magStrategy := mag.MagStrategy(magSampler, mag.BatchSize, mag.TrainSplit)


fmt.Printf("Elapsed: %s\n", time.Since(start))
fmt.Printf("\n%s\n", magSampler)
fmt.Printf("\n%s\n", magStrategy)

Elapsed: 942.678854ms

Sampler: 4 node types, 8 edge types, Frozen
	NodeType "papers": 736,389 items
	NodeType "authors": 1,134,649 items
	NodeType "institutions": 8,740 items
	NodeType "fields_of_study": 59,965 items
	EdgeType "writtenBy": ["papers"]->["authors"], 7,145,660 edges
	EdgeType "cites": ["papers"]->["papers"], 5,416,271 edges
	EdgeType "citedBy": ["papers"]->["papers"], 5,416,271 edges
	EdgeType "affiliatedWith": ["authors"]->["institutions"], 1,043,998 edges
	EdgeType "affiliations": ["institutions"]->["authors"], 1,043,998 edges
	EdgeType "hasTopic": ["papers"]->["fields_of_study"], 7,505,078 edges
	EdgeType "topicHasPapers": ["fields_of_study"]->["papers"], 7,505,078 edges
	EdgeType "writes": ["authors"]->["papers"], 7,145,660 edges

Sampling strategy: (13 Rules)
> Rule "seeds": type=Node, nodeType="papers", Shape=(Int32)[128] (size=128), NodeSet.size=629571
  > Rule "citations": type=Edge, nodeType="papers", Shape=(Int32)[128 8] (size=1024), SourceRule="seeds", EdgeTyp

## FNN model, No Graph

The first model will only use the paper features, and no relations. It serves as a baseline.

In a quick experiment, without much hyperparameter tuning, we got 27.27% on test accuracy, which is inline with the corresponding results in [the leaderboard](https://ogb.stanford.edu/docs/leader_nodeprop/#ogbn-mag) (the Multi Layer Perceptron MLP entry in the bottom).

In [4]:
func config() (ctx *context.Context) {
    ctx = context.NewContext(manager)
    ctx.RngStateReset()
    ctx.SetParams(map[string]any{
        "train_steps": 1_000_000, 
        "batch_size": 128,
        "optimizer": "adamw", 
        optimizers.LearningRateKey: 0.0001,
        layers.L2RegularizationKey: 1e-4,
        "normalization": "layer",
        "dropout": 0.1,
        "hidden_layers": 2,
        "num_nodes": 256,
        "plots": true,
    })
    return
}

In [6]:
import "github.com/gomlx/gomlx/examples/ogbnmag/fnn"

%%
must.M(mag.Download(*flagDataDir))
ctx := config()
ctx.SetParam("checkpoint", path.Join(*flagDataDir, "fnn-baseline"))
ctx.SetParam("num_checkpoints", 10)
err := fnn.Train(ctx)
if  err != nil {
    fmt.Printf("%+v\n", err)
}


> papers features input shape: (Float32)[128 129]
Training (1000000 steps):  100% [========================================] (2052 steps/s) [loss=2.793] [~loss=3.078] [~acc=28.30%]           


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,4.642021e+01 L8.604205e+01,2.461840e+01 L1.279667e+02,4.339416e+01 L1.590885e+02,7.031880e+01 L1.842956e+02,7.441706e+01 L2.058775e+02,9.362171e+01 L2.249197e+02,1.185210e+02 L2.421551e+02,1.350571e+02 L2.580360e+02,1.416572e+02 L2.728577e+02,1.554292e+02 L2.868215e+02,1.685818e+02 L3.000697e+02,1.643644e+02 L3.127433e+02,1.745624e+02 L3.249391e+02,1.709169e+02 L3.367242e+02,1.821633e+02 L3.481462e+02,1.730137e+02 L3.592621e+02,1.843315e+02 L3.701096e+02,1.883345e+02 L3.807317e+02,1.956971e+02 L3.911552e+02,1.921945e+02 L4.014103e+02,2.001766e+02 L4.115139e+02,2.034647e+02 L4.214856e+02,1.985801e+02 L4.313350e+02,2.094691e+02 L4.410740e+02,2.093262e+02 L4.507148e+02,1.984848e+02 L4.602692e+02,2.057283e+02 L4.697471e+02,2.177610e+02 L4.791566e+02,2.228123e+02 L4.885037e+02,2.072532e+02 L4.977974e+02,2.206202e+02 L5.070430e+02,2.240275e+02 L5.162428e+02,2.218592e+02 L5.254041e+02,2.143537e+02 L5.345295e+02,2.088973e+02 L5.436214e+02,2.125190e+02 L5.526839e+02,2.137580e+02 L5.617192e+02,2.267914e+02 L5.707319e+02,2.197624e+02 L5.797236e+02,2.239798e+02 L5.886968e+02,2.285308e+02 L5.976522e+02,2.124237e+02 L6.065921e+02,2.238607e+02 L6.155178e+02,2.251712e+02 L6.244301e+02,2.185949e+02 L6.333308e+02,2.258383e+02 L6.422211e+02,2.360840e+02 L6.511014e+02,2.178324e+02 L6.599729e+02,2.266723e+02 L6.688363e+02,2.273156e+02 L6.776927e+02,2.255763e+02 L6.865421e+02,2.308182e+02 L6.953852e+02,2.309612e+02 L7.042229e+02,2.357504e+02 L7.130556e+02,2.355122e+02 L7.218836e+02,2.256716e+02 L7.307073e+02,2.356313e+02 L7.395271e+02,2.384667e+02 L7.483437e+02,2.358219e+02 L7.571570e+02,2.334392e+02 L7.659676e+02,2.361079e+02 L7.747756e+02,2.333677e+02 L7.835814e+02,2.408256e+02 L7.923849e+02,2.381331e+02 L8.011865e+02,2.429224e+02 L8.099864e+02,2.345114e+02 L8.187845e+02,2.349880e+02 L8.275813e+02,2.367750e+02 L8.363766e+02,2.459246e+02 L8.451708e+02,2.370133e+02 L8.539638e+02,2.372515e+02 L8.627558e+02,2.315569e+02 L8.663200e+02,2.438040e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,3.459487e+01 L8.604205e+01,6.310184e+01 L1.279667e+02,7.803943e+01 L1.590885e+02,9.598168e+01 L1.842956e+02,1.014593e+02 L2.058775e+02,1.128113e+02 L2.249197e+02,1.206078e+02 L2.421551e+02,1.248632e+02 L2.580360e+02,1.323614e+02 L2.728577e+02,1.375977e+02 L2.868215e+02,1.460070e+02 L3.000697e+02,1.515639e+02 L3.127433e+02,1.556352e+02 L3.249391e+02,1.629572e+02 L3.367242e+02,1.665237e+02 L3.481462e+02,1.698093e+02 L3.592621e+02,1.743663e+02 L3.701096e+02,1.784059e+02 L3.807317e+02,1.802804e+02 L3.911552e+02,1.827264e+02 L4.014103e+02,1.891499e+02 L4.115139e+02,1.898563e+02 L4.214856e+02,1.932577e+02 L4.313350e+02,1.945720e+02 L4.410740e+02,1.978592e+02 L4.507148e+02,1.977671e+02 L4.602692e+02,2.013511e+02 L4.697471e+02,2.023939e+02 L4.791566e+02,2.050446e+02 L4.885037e+02,2.077080e+02 L4.977974e+02,2.081795e+02 L5.070430e+02,2.101445e+02 L5.162428e+02,2.121254e+02 L5.254041e+02,2.135364e+02 L5.345295e+02,2.135904e+02 L5.436214e+02,2.156030e+02 L5.526839e+02,2.180204e+02 L5.617192e+02,2.186727e+02 L5.707319e+02,2.205457e+02 L5.797236e+02,2.218441e+02 L5.886968e+02,2.226837e+02 L5.976522e+02,2.233123e+02 L6.065921e+02,2.249995e+02 L6.155178e+02,2.262296e+02 L6.244301e+02,2.272502e+02 L6.333308e+02,2.290216e+02 L6.422211e+02,2.297629e+02 L6.511014e+02,2.303168e+02 L6.599729e+02,2.323247e+02 L6.688363e+02,2.336342e+02 L6.776927e+02,2.353357e+02 L6.865421e+02,2.352389e+02 L6.953852e+02,2.371737e+02 L7.042229e+02,2.383372e+02 L7.130556e+02,2.385895e+02 L7.218836e+02,2.387102e+02 L7.307073e+02,2.407609e+02 L7.395271e+02,2.416815e+02 L7.483437e+02,2.422370e+02 L7.571570e+02,2.435354e+02 L7.659676e+02,2.436782e+02 L7.747756e+02,2.449338e+02 L7.835814e+02,2.469559e+02 L7.923849e+02,2.468654e+02 L8.011865e+02,2.477924e+02 L8.099864e+02,2.481844e+02 L8.187845e+02,2.499288e+02 L8.275813e+02,2.495669e+02 L8.363766e+02,2.504891e+02 L8.451708e+02,2.513351e+02 L8.539638e+02,2.521494e+02 L8.627558e+02,2.5209

<path d="M1.768000e+01,2.471893e+02 L8.604205e+01,2.138338e+02 L1.279667e+02,1.967233e+02 L1.590885e+02,1.755666e+02 L1.842956e+02,1.555239e+02 L2.058775e+02,1.375292e+02 L2.249197e+02,1.215023e+02 L2.421551e+02,1.107117e+02 L2.580360e+02,9.945654e+01 L2.728577e+02,9.112112e+01 L2.868215e+02,8.174477e+01 L3.000697e+02,7.715769e+01 L3.127433e+02,7.264858e+01 L3.249391e+02,6.781227e+01 L3.367242e+02,6.351467e+01 L3.481462e+02,6.513197e+01 L3.592621e+02,5.645136e+01 L3.701096e+02,5.501699e+01 L3.807317e+02,5.059189e+01 L3.911552e+02,4.788816e+01 L4.014103e+02,4.747247e+01 L4.115139e+02,4.323816e+01 L4.214856e+02,4.175481e+01 L4.313350e+02,3.794835e+01 L4.410740e+02,3.620850e+01 L4.507148e+02,3.568142e+01 L4.602692e+02,3.489394e+01 L4.697471e+02,2.980806e+01 L4.791566e+02,2.845564e+01 L4.885037e+02,2.996045e+01 L4.977974e+02,2.758061e+01 L5.070430e+02,2.442668e+01 L5.162428e+02,2.411016e+01 L5.254041e+02,2.557521e+01 L5.345295e+02,2.409235e+01 L5.436214e+02,2.376144e+01 L5.526839e+02,2.260846e+01 L5.617192e+02,1.892723e+01 L5.707319e+02,1.911039e+01 L5.797236e+02,1.861303e+01 L5.886968e+02,1.816060e+01 L5.976522e+02,1.840698e+01 L6.065921e+02,1.678321e+01 L6.155178e+02,1.576176e+01 L6.244301e+02,1.692495e+01 L6.333308e+02,1.540866e+01 L6.422211e+02,1.279581e+01 L6.511014e+02,1.610782e+01 L6.599729e+02,1.406931e+01 L6.688363e+02,1.330928e+01 L6.776927e+02,1.403999e+01 L6.865421e+02,1.387263e+01 L6.953852e+02,1.321742e+01 L7.042229e+02,1.187967e+01 L7.130556e+02,9.740832e+00 L7.218836e+02,1.292048e+01 L7.307073e+02,1.159137e+01 L7.395271e+02,9.676964e+00 L7.483437e+02,1.267576e+01 L7.571570e+02,1.126766e+01 L7.659676e+02,1.041291e+01 L7.747756e+02,1.084019e+01 L7.835814e+02,1.002294e+01 L7.923849e+02,1.190406e+01 L8.011865e+02,8.559848e+00 L8.099864e+02,1.066062e+01 L8.187845e+02,1.142621e+01 L8.275813e+02,9.740398e+00 L8.363766e+02,9.824061e+00 L8.451708e+02,9.768421e+00 L8.539638e+02,1.090919e+01 L8.627558e+02,1.154322e+01 L8.663200e+02,8.757371e+00 " vector-effect="non-scaling-stroke"/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.507164e+02 L8.604205e+01,2.222679e+02 L1.279667e+02,2.074635e+02 L1.590885e+02,1.892033e+02 L1.842956e+02,1.710122e+02 L2.058775e+02,1.546667e+02 L2.249197e+02,1.399427e+02 L2.421551e+02,1.272186e+02 L2.580360e+02,1.161804e+02 L2.728577e+02,1.067309e+02 L2.868215e+02,9.869594e+01 L3.000697e+02,9.169229e+01 L3.127433e+02,8.561225e+01 L3.249391e+02,7.967401e+01 L3.367242e+02,7.530824e+01 L3.481462e+02,7.116764e+01 L3.592621e+02,6.691199e+01 L3.701096e+02,6.338275e+01 L3.807317e+02,5.999705e+01 L3.911552e+02,5.722928e+01 L4.014103e+02,5.414340e+01 L4.115139e+02,5.154301e+01 L4.214856e+02,4.846362e+01 L4.313350e+02,4.615061e+01 L4.410740e+02,4.374663e+01 L4.507148e+02,4.180144e+01 L4.602692e+02,3.950532e+01 L4.697471e+02,3.774124e+01 L4.791566e+02,3.567653e+01 L4.885037e+02,3.394415e+01 L4.977974e+02,3.238433e+01 L5.070430e+02,3.073251e+01 L5.162428e+02,2.950235e+01 L5.254041e+02,2.803988e+01 L5.345295e+02,2.696244e+01 L5.436214e+02,2.570268e+01 L5.526839e+02,2.438342e+01 L5.617192e+02,2.349525e+01 L5.707319e+02,2.222555e+01 L5.797236e+02,2.123247e+01 L5.886968e+02,2.045457e+01 L5.976522e+02,1.948113e+01 L6.065921e+02,1.853162e+01 L6.155178e+02,1.760553e+01 L6.244301e+02,1.681297e+01 L6.333308e+02,1.626097e+01 L6.422211e+02,1.526453e+01 L6.511014e+02,1.501348e+01 L6.599729e+02,1.392143e+01 L6.688363e+02,1.339992e+01 L6.776927e+02,1.280627e+01 L6.865421e+02,1.211152e+01 L6.953852e+02,1.157032e+01 L7.042229e+02,1.092431e+01 L7.130556e+02,1.072641e+01 L7.218836e+02,1.026321e+01 L7.307073e+02,9.959726e+00 L7.395271e+02,9.400204e+00 L7.483437e+02,8.962009e+00 L7.571570e+02,8.507820e+00 L7.659676e+02,8.203253e+00 L7.747756e+02,7.874464e+00 L7.835814e+02,7.495195e+00 L7.923849e+02,7.358715e+00 L8.011865e+02,6.993403e+00 L8.099864e+02,6.743716e+00 L8.187845e+02,6.302398e+00 L8.275813e+02,6.273424e+00 L8.363766e+02,5.976053e+00 L8.451708e+02,5.742550e+00 L8.539638e+02,5.510296e+00 L8.627558e+02,5.2139

<path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.548000e+02 L8.604205e+01,2.165992e+02 L1.279667e+02,1.930165e+02 L1.590885e+02,1.750276e+02 L1.842956e+02,1.559756e+02 L2.058775e+02,1.425977e+02 L2.249197e+02,1.285271e+02 L2.421551e+02,1.166869e+02 L2.580360e+02,1.044894e+02 L2.728577e+02,9.608524e+01 L2.868215e+02,8.538934e+01 L3.000697e+02,8.203044e+01 L3.127433e+02,7.529248e+01 L3.249391e+02,7.173424e+01 L3.367242e+02,6.563624e+01 L3.481462e+02,6.140053e+01 L3.592621e+02,5.829147e+01 L3.701096e+02,5.545259e+01 L3.807317e+02,5.369662e+01 L3.911552e+02,5.071524e+01 L4.014103e+02,4.742105e+01 L4.115139e+02,4.685687e+01 L4.214856e+02,4.399400e+01 L4.313350e+02,4.380308e+01 L4.410740e+02,3.937225e+01 L4.507148e+02,3.875468e+01 L4.602692e+02,3.549824e+01 L4.697471e+02,3.397336e+01 L4.791566e+02,3.217121e+01 L4.885037e+02,3.158926e+01 L4.977974e+02,2.708378e+01 L5.070430e+02,2.839203e+01 L5.162428e+02,2.709628e+01 L5.254041e+02,2.498603e+01 L5.345295e+02,2.420268e+01 L5.436214e+02,2.475445e+01 L5.526839e+02,2.151766e+01 L5.617192e+02,2.070694e+01 L5.707319e+02,1.811688e+01 L5.797236e+02,1.907704e+01 L5.886968e+02,1.920607e+01 L5.976522e+02,1.730337e+01 L6.065921e+02,1.661154e+01 L6.155178e+02,1.702512e+01 L6.244301e+02,1.588820e+01 L6.333308e+02,1.547385e+01 L6.422211e+02,1.490165e+01 L6.511014e+02,1.427546e+01 L6.599729e+02,1.321109e+01 L6.688363e+02,1.547975e+01 L6.776927e+02,1.124671e+01 L6.865421e+02,1.299568e+01 L6.953852e+02,1.091305e+01 L7.042229e+02,1.017335e+01 L7.130556e+02,1.108588e+01 L7.218836e+02,9.718072e+00 L7.307073e+02,1.139137e+01 L7.395271e+02,1.006325e+01 L7.483437e+02,7.832119e+00 L7.571570e+02,8.878879e+00 L7.659676e+02,8.939918e+00 L7.747756e+02,6.057936e+00 L7.835814e+02,7.546156e+00 L7.923849e+02,7.097069e+00 L8.011865e+02,9.233878e+00 L8.099864e+02,8.495800e+00 L8.187845e+02,7.565801e+00 L8.275813e+02,6.946981e+00 L8.363766e+02,6.196607e+00 L8.451708e+02,6.051141e+00 L8.539638e+02,5.637855e+00 L8.627558e+02,7.302141e+00 L8.663200e+02,5.200000e+00 "/> <path d="M1.768000e+01,0 L1.768000e+01,-6 M8.154652e+01,0 L8.154652e+01,-6 M1.189060e+02,0 L1.189060e+02,-6 M1.454130e+02,0 L1.454130e+02,-6 M1.659735e+02,0 L1.659735e+02,-6 M1.827726e+02,0 L1.827726e+02,-6 M1.969760e+02,0 L1.969760e+02,-6 M2.092796e+02,0 L2.092796e+02,-6 M2.201321e+02,0 L2.201321e+02,-6 M2.298400e+02,0 L2.298400e+02,-6 M2.386219e+02,0 L2.386219e+02,-6 M2.937065e+02,0 L2.937065e+02,-6 M3.310660e+02,0 L3.310660e+02,-6 M3.575730e+02,0 L3.575730e+02,-6 M3.781335e+02,0 L3.781335e+02,-6 M3.949326e+02,0 L3.949326e+02,-6 M4.091360e+02,0 L4.091360e+02,-6 M4.214396e+02,0 L4.214396e+02,-6 M4.322921e+02,0 L4.322921e+02,-6 M442,0 L442,-6 M5.058665e+02,0 L5.058665e+02,-6 M5.432260e+02,0 L5.432260e+02,-6 M5.697330e+02,0 L5.697330e+02,-6 M5.902935e+02,0 L5.902935e+02,-6 M6.070926e+02,0 L6.070926e+02,-6 M6.212960e+02,0 L6.212960e+02,-6 M6.335996e+02,0 L6.335996e+02,-6 M6.444521e+02,0 L6.444521e+02,-6 M6.541600e+02,0 L6.541600e+02,-6 M7.180265e+02,0 L7.180265e+02,-6 M7.553860e+02,0 L7.553860e+02,-6 M7.818930e+02,0 L7.818930e+02,-6 M8.024535e+02,0 L8.024535e+02,-6 M8.192526e+02,0 L8.192526e+02,-6 M8.334560e+02,0 L8.334560e+02,-6 M8.457596e+02,0 L8.457596e+02,-6 M8.566121e+02,0 L8.566121e+02,-6 M8.663200e+02,0 L8.663200e+02,-6 " vector-effect="non-scaling-stroke"/> 100 200 300 400 500 600 700 800 1000 2000 3000 4000 5000 6000 7000 8000 10000 20000 30000 40000 50000 60000 70000 80000 100000 200000 300000 400000 500000 600000 700000 800000 1000000 Steps 4.000 5.000 loss loss metrics Train: Moving Average Loss

	[Step 1000000] median train step: 407 microseconds

Results on seeds_train:
	Mean Loss (#loss): 3.083
	Mean Accuracy (#acc): 28.07%
Results on seeds_valid:
	Mean Loss (#loss): 3.186
	Mean Accuracy (#acc): 26.28%
Results on seeds_test:
	Mean Loss (#loss): 3.114
	Mean Accuracy (#acc): 27.20%



## GNN (Graph Neural Networks)

To use GNNs we need first graphs to run the model on. The issue is, generally the graphs in real life are too large (social networks, relational datases, etc.), including OGNB-MAG.

So instead we use sampled sub-graphs to train. For inference we can also use sampled subgraphs, but later we show a work around where we can do inference, a node type at a time, where we don't need sampling.

### Sampling Sub-Graphs

We follow the same sampling strategy used in [TensorFlow GNN](https://github.com/tensorflow/gnn) library, describe it its [OGBN-MAG notebook](https://github.com/tensorflow/gnn/blob/main/examples/notebooks/ogbn_mag_e2e.ipynb).

In [7]:
import "github.com/gomlx/gomlx/examples/ogbnmag/sampler"

// MagStrategy takes a sampler created by [ogbnmag.NewSampler], a desired batch size, and the set of
// seed ids to sample from ([ogbnmag.TrainSplit], [ogbnmag.ValidSplit] or [ogbnmag.TestSplit]) and
// returns a sampling strategy, that can be used to create datasets.
func MagStrategy(magSampler *sampler.Sampler, batchSize int, seedIdsCandidates tensor.Tensor) *sampler.Strategy {
	strategy := magSampler.NewStrategy()
	var seeds *sampler.Rule
	if seedIdsCandidates == nil {
		seeds = strategy.Nodes("seeds", "papers", batchSize)
	} else {
		seedIdsData := seedIdsCandidates.Local().FlatCopy().([]int32)
		seeds = strategy.NodesFromSet("seeds", "papers", batchSize, seedIdsData)
	}
	citations := seeds.FromEdges("citations", "cites", 8)

	// Authors
	seedsAuthors := seeds.FromEdges("seedsAuthors", "writtenBy", 8)
	citationsAuthors := citations.FromEdges("citationsAuthors", "writtenBy", 8)

	// Co-authored papers
	coauthoredPapers := seedsAuthors.FromEdges("coauthoredPapers", "writes", 8)
	coauthoredFromCitations := citationsAuthors.FromEdges("coauthoredFromCitations", "writes", 8)

	// Affiliations
	_ = seedsAuthors.FromEdges("authorsInstitutions", "affiliatedWith", 8)
	_ = citationsAuthors.FromEdges("citationAuthorsInstitutions", "affiliatedWith", 8)

	// Topics
	_ = seeds.FromEdges("seedsTopics", "hasTopic", 8)
	_ = coauthoredPapers.FromEdges("coauthoredTopics", "hasTopic", 8)
	_ = citations.FromEdges("citationsTopics", "hasTopic", 8)
	_ = coauthoredFromCitations.FromEdges("coauthoredFromCitationsTopics", "hasTopic", 8)

	return strategy
}


### Benchmarking Sampling

Let's get some benchmarking on the sampler, to make sure it's not slowing training:

1. Benchmark sequential the sampling of one sub-graph: `BenchmarkSequentialMagSampler`
2. Benchmark parallel sampling of one sub-graph: `BenchmarkParallelMagSampler`
3. Benchmark parallel sampling of one epoch (using parallel sampler): `BenchmarkOneEpochMagSampler`

### Training Model

We use the vanilla GNN trainer and model defined in the `gnn` package.

In [74]:
import (
    "github.com/gomlx/gomlx/examples/ogbnmag/gnn"
)

func configGnn(baseDir string) *context.Context {
    must.M(mag.Download(baseDir))
    ctx := context.NewContext(manager)
    ctx.RngStateReset()
    
    stepsPerEpoch := mag.TrainSplit.Shape().Size() / mag.BatchSize + 1
    numEpochs := 10  // Taken from TF-GNN OGBN-MAG notebook.
    numTrainSteps := numEpochs * stepsPerEpoch
    
    ctx.SetParams(map[string]any{
        "train_steps": numTrainSteps,
        
        optimizers.ParamOptimizer: "adam", 
        optimizers.ParamLearningRate: 0.001,
        optimizers.ParamCosineScheduleSteps:  numTrainSteps,
        
        layers.ParamL2Regularization: 1e-5,
        layers.ParamDropoutRate: 0.2,

        mag.ParamEmbedDropoutRate: 0.0,
        
        gnn.ParamEdgeDropoutRate: 0.0,
        gnn.ParamNumGraphUpdates: 2,
        gnn.ParamReadoutHiddenLayers: 2,
        gnn.ParamPoolingType: "mean|sum",
        gnn.ParamUsePathToRootStates: false,
        
        "plots": true,
    })
    return ctx
}

%%
mag.BatchSize = 128  // Default is 128.
ctx := configGnn(*flagDataDir)
// ctx.SetParam("checkpoint", path.Join(*flagDataDir, "gnn-small_batch"))
// ctx.SetParam("checkpoint", path.Join(*flagDataDir, "gnn-use_path_to_root"))
ctx.SetParam("checkpoint", path.Join(*flagDataDir, "gnn-reg_2"))
ctx.SetParam("num_checkpoints", 3)
ctx.SetParams(map[string]any{
    // "train_steps": 100,
    // "plots": false,
    // gnn.ParamEdgeDropoutRate: 0.1,
    // mag.ParamEmbedDropoutRate: 0.1,
    gnn.ParamNumGraphUpdates: 2,
    // gnn.ParamUsePathToRootStates: true,
    // gnn.ParamReadoutHiddenLayers: 2,
    layers.ParamDropoutRate: 0.25,
    layers.ParamL2Regularization: 3e-4,
})

// err := gnn.Train(ctx, *flagDataDir)
// if  err != nil {
//     fmt.Printf("%+v\n", err)
// }

In [14]:
%%
ctx := configGnn(*flagDataDir)
ctx.SetParam("checkpoint", path.Join(*flagDataDir, "gnn-baseline_5"))

_, trainEvalDS, validEvalDS, testEvalDS := must.M4(gnn.MakeDatasets(*flagDataDir))
_, _, _ = trainEvalDS, validEvalDS, testEvalDS
must.M(gnn.Eval(ctx, *flagDataDir, trainEvalDS, validEvalDS, testEvalDS))

loading: "checkpoint-n0000171-20240229-012409-step-00049190"
Model in "/home/janpf/work/ogbnmag/gnn-baseline_5" trained for 49190 steps.
Results on train:
	Mean Loss+Regularization (#loss+): 1.542
	Mean Loss (#loss): 1.457
	Mean Accuracy (#acc): 57.49%
	elapsed 1m37.457905276s (train)
Results on valid:
	Mean Loss+Regularization (#loss+): 1.927
	Mean Loss (#loss): 1.841
	Mean Accuracy (#acc): 48.60%
	elapsed 11.140775268s (valid)
Results on test:
	Mean Loss+Regularization (#loss+): 1.922
	Mean Loss (#loss): 1.837
	Mean Accuracy (#acc): 48.10%
	elapsed 7.647081029s (test)


In [122]:
%%
ctx := configGnn(*flagDataDir)
ctx.SetParam("checkpoint", path.Join(*flagDataDir, "gnn-baseline_17"))

_, trainEvalDS, validEvalDS, testEvalDS := must.M4(mag.MakeDatasets(*flagDataDir))
_, _, _ = trainEvalDS, validEvalDS, testEvalDS
must.M(mag.Eval(ctx, *flagDataDir, validEvalDS, testEvalDS))

loading: "checkpoint-n0000142-20240305-235708-step-00049190"
Model in "/home/janpf/work/ogbnmag/gnn-baseline_17" trained for 49190 steps.
Results on valid:
	Mean Loss+Regularization (#loss+): 1.920
	Mean Loss (#loss): 1.834
	Mean Accuracy (#acc): 48.64%
	elapsed 9.335698266s (valid)
Results on test:
	Mean Loss+Regularization (#loss+): 1.917
	Mean Loss (#loss): 1.830
	Mean Accuracy (#acc): 48.38%
	elapsed 5.863748141s (test)


In [8]:
import (
    "github.com/gomlx/gomlx/examples/notebook/gonb/plots"
    "github.com/janpfeifer/gonb/gonbui/plotly"
    grob "github.com/MetalBlueberry/go-plotly/graph_objects"
)

func renamePoint(p *plots.Point) {
    p.MetricName = strings.Replace(p.MetricName, "Eval on ", "", -1)
    
    p.MetricName = strings.Replace(p.MetricName, "Accuracy", "acc", -1)
    p.MetricName = strings.Replace(p.MetricName, "Loss+Regularization", "loss+", -1)
    p.MetricName = strings.Replace(p.MetricName, "Loss", "loss", -1)
    
    p.MetricName = strings.Replace(p.MetricName, " Mean ", "#", -1)
    p.MetricName = strings.Replace(p.MetricName, " Moving Average ", "~", -1)
}

func FilterPoints(points plots.Points) {
    points.Map(renamePoint)
    batchSize := float64(4919)
    _ = batchSize
    points.Filter(func (p plots.Point) bool {
        return strings.Index(p.MetricName, "Batch") < 0 &&
            // strings.Index(p.MetricName, "~") < 0 &&
            strings.Index(p.MetricName, "Train:~loss") < 0 &&
            strings.Index(p.MetricName, "Train:~acc") < 0 &&
            strings.Index(p.MetricName, "loss+") < 0 &&
            // math.Abs(math.Round(p.Step / batchSize)*batchSize - p.Step) < 10 &&
            true
    })
}

%%
checkpoints := []string{"gnn-baseline_24", "gnn-baseline_26"}
prefixes := []string{"(A)", "(B)"}

var merged plots.Points
for ii, checkpoint := range checkpoints {
    pts := plots.NewPoints(must.M1(plots.LoadPointsFromCheckpoint(path.Join(*flagDataDir, checkpoint))))
    FilterPoints(pts)
    pts.Map(func(p *plots.Point) {
       p.MetricName = prefixes[ii] + p.MetricName 
    })
    
    if merged == nil {
        merged = pts
    } else {
        merged.Add(pts)
    }
}

fmt.Printf("%s\n", merged)

╭───────┬───────────────┬───────────────┬───────────────┬───────────────┬────────────────┬────────────────┬────────────────┬────────────────╮
│ Step  │ (A)train:#acc │ (A)valid:#acc │ (B)train:#acc │ (B)valid:#acc │ (A)train:#loss │ (A)valid:#loss │ (B)train:#loss │ (B)valid:#loss │
├───────┼───────────────┼───────────────┼───────────────┼───────────────┼────────────────┼────────────────┼────────────────┼────────────────┤
│ 200   │ 0.180916      │ 0.171514      │ 0.166444      │ 0.122666      │ 3.773040       │ 3.754423       │ 3.932029       │ 4.029837       │
│ 440   │ 0.247725      │ 0.240095      │ 0.228974      │ 0.250166      │ 3.233717       │ 3.228036       │ 3.304488       │ 3.325431       │
│ 728   │ 0.279135      │ 0.266171      │ 0.276081      │ 0.279186      │ 2.960663       │ 2.991662       │ 2.985939       │ 3.023475       │
│ 1074  │ 0.305296      │ 0.280638      │ 0.302901      │ 0.301041      │ 2.745518       │ 2.798317       │ 2.795963       │ 2.841690       │
│ 1489

In [15]:
import (
    stdplots "github.com/gomlx/gomlx/examples/notebook/gonb/plots"
    "github.com/gomlx/gomlx/examples/notebook/gonb/plotly"
)
%%
versions := []int{71, 72}

plots := plotly.New()
for _, version := range versions {
    // checkpoint := fmt.Sprintf("gnn-baseline_%d", version)
    checkpoint := fmt.Sprintf("small_%d", version)
    prefix := fmt.Sprintf("(%d) ", version)
    must.M(plots.LoadCheckpointData(path.Join(*flagDataDir, checkpoint), 
        func (pt *stdplots.Point) bool {
            renamePoint(pt)
            pt.MetricName = prefix+pt.MetricName
            return strings.Index(pt.MetricName, "Batch") < 0 &&
                // strings.Index(pt.MetricName, "~") < 0 &&
                strings.Index(pt.MetricName, "Train:~loss") < 0 &&
                strings.Index(pt.MetricName, "Train:~acc") < 0 &&
                strings.Index(pt.MetricName, "loss+") < 0 &&
                // math.Abs(math.Round(pt.Step / batchSize)*batchSize - pt.Step) < 10 &&
                true
        }))
}

// plots.Dynamic()
// plots.DynamicPlot(false)
plots.Plot()

Metric: accuracy

Metric: loss

### Layer-wise Inference Experiments

In [25]:
import (
    mldata "github.com/gomlx/gomlx/ml/data"
    "github.com/gomlx/gomlx/ml/context/checkpoints"
)

func loadCtx(checkpointName string) (ctx *context.Context) {
    ctx = context.NewContext(manager)
	mag.UploadOgbnMagVariables(ctx)

    // Exclude from saving all the variables created by the `mag` package -- specially the frozen papers embeddings,
    // which take most space.
    var varsToExclude []*context.Variable
    ctx.InAbsPath(mag.OgbnMagVariablesScope).EnumerateVariablesInScope(func(v *context.Variable) {
        varsToExclude = append(varsToExclude, v)
    })
    _ = must.M1(checkpoints.
                Build(ctx).
                DirFromBase(checkpointName, *flagDataDir).
                ExcludeVarsFromSaving(varsToExclude...).
                Done())
    return ctx
}

%%
must.M(mag.Download(*flagDataDir))
magSampler := must.M1(mag.NewSampler(*flagDataDir))
magStrategy := mag.MagStrategy(magSampler, mag.BatchSize, nil)
ctx := loadCtx("samll_71")
_ = mag.LayerWiseInference(ctx, magStrategy)

fmt.Println("Done.")

Rule "citations": NumNodes=736389
Rule "citationsAuthors": NumNodes=1134649
Rule "papersByCitationAuthors": NumNodes=736389
Rule "papersByCitationAuthorsTopics": NumNodes=59965
Rule "citationAuthorsInstitutions": NumNodes=8740
Rule "citationsTopics": NumNodes=59965
Rule "seedsBase": NumNodes=736389
Rule "seedsAuthors": NumNodes=1134649
Rule "papersByAuthors": NumNodes=736389
Rule "papersByAuthorsTopics": NumNodes=59965
Rule "authorsInstitutions": NumNodes=8740
Rule "seedsTopics": NumNodes=59965


panic: Missing OGBN-MAG dataset variables ("EdgeWrites"), pls call UploadOgbnMagVariables() on context first.

goroutine 1 [running]:
github.com/gomlx/exceptions.Panicf(...)
	/home/janpf/src/go/pkg/mod/github.com/gomlx/exceptions@v0.0.3/exceptions.go:92
github.com/gomlx/gomlx/examples/ogbnmag.getMagVar(0xc0001a02a0?, 0xc00017a900, {0xc00012c8e0, 0xa})
	/home/janpf/Projects/gomlx/examples/ogbnmag/model.go:30 +0xae
github.com/gomlx/gomlx/examples/ogbnmag.createEdgesIndices(0xc0000d0900, 0xc00017a900)
	/home/janpf/Projects/gomlx/examples/ogbnmag/lwinference.go:105 +0x159
github.com/gomlx/gomlx/examples/ogbnmag.createEdgesInputs(0xc00017a900?, 0xc00017a900, 0xc0000d0180, {0xc00017e248, 0x26, 0x41})
	/home/janpf/Projects/gomlx/examples/ogbnmag/lwinference.go:94 +0x36
github.com/gomlx/gomlx/examples/ogbnmag.LayerWiseInference.BuildLayerWiseInferenceModel.func1(0xc0000d0240, 0xc00017a900)
	/home/janpf/Projects/gomlx/examples/ogbnmag/lwinference.go:44 +0xc8
reflect.Value.call({0x7540c0?, 0xc00